In [1]:
%cd "C:\\Users\\budin\\Desktop\\github\\reinforcement learning trading commodity futures\\"
path = "C:\\Users\\budin\\Desktop\\github\\reinforcement learning trading commodity futures\\"

import warnings
warnings.filterwarnings("ignore")
import time
from tqdm import tqdm
import numpy as np
import pandas as pd
#显示Dateframe所有列(参数设置为None代表显示所有行，也可以自行设置数字)
pd.set_option('display.max_columns',None)
#显示Dateframe所有行
pd.set_option('display.max_rows',None)
#设置Dataframe数据的显示长度，默认为50
pd.set_option('max_colwidth',200)
#不显示科学计数法，保留5位小数
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import random
from env.futures_portfoilo_env import *
from stable_baselines3 import *

import torch
import os
import datetime

C:\Users\budin\Desktop\github\reinforcement learning trading commodity futures


# Read Data

In [2]:
train_obs = np.load('Processed Data//processed_train.npy', allow_pickle=True)
train_symbols = np.load('Processed Data//processed_train_1.npy', allow_pickle=True)
train_price = np.load('Processed Data//processed_train_2.npy', allow_pickle=True)
stock_num = len(train_symbols[0])-1

train_obs.shape, train_price.shape, train_symbols.shape, train_price[0], train_price[-1], stock_num

((8131, 147, 36),
 (8131, 6),
 (8131, 2),
 array(['4687.0', '4704.0', '4651.0', '4675.0', '214037',
        '2021-03-01 09:15:00'], dtype='<U32'),
 array(['4115.0', '4120.0', '4110.0', '4115.0', '36422',
        '2022-08-15 23:00:00'], dtype='<U32'),
 1)

In [3]:
train_symbols[0], np.where(np.isnan(train_obs)==True), np.where(np.isinf(train_obs)==True)

(array(['SHFE.rb2105', '2021-03-01 09:15:00'], dtype='<U19'),
 (array([], dtype=int64), array([], dtype=int64), array([], dtype=int64)),
 (array([], dtype=int64), array([], dtype=int64), array([], dtype=int64)))

# Data Quality Check
If the training dataset contains too many zeros, it may negatively affect the resulting model's accuracy.

In [4]:
zeros = []
names = []
columns = ['open', 'close', 'high', 'low', 'volume', 'symbol', 'DX', 'CORREL', 'HT_PHASOR1', 'SUB', 'MINUS_DM', 'TRANGE', 'MINUS_DI', 'DIV',
            'ATR', 'AD', 'MAX', 'MINMAX1', 'LINEARREG_INTERCEPT', 'ADX', 'TRIMA', 'NATR', 'STDDEV',
            'HT_TRENDLINE', 'KAMA', 'SUM', 'SMA', 'MA', 'SAR', 'WMA', 'EMA', 'MIDPRICE', 'MAMA1',
            'MIDPOINT', 'DEMA', 'BBANDS0']
for i in range(train_obs.shape[-1]):
    tmp_zeros = (train_obs[np.where(train_obs[:,:,i]==0)].shape[0])
    names.append(columns[i])
    zeros.append(tmp_zeros)
    
data_quality = pd.DataFrame()
names.append('Total Zeros')
zeros.append(train_obs[np.where(train_obs==0)].shape[0])
names.append('Total Zeros Pct')
zeros.append((train_obs[np.where(train_obs==0)].shape[0])/(train_obs.flatten().shape[0]))
data_quality['name'] = names
data_quality['zeros'] = zeros

data_quality

,name,zeros
0,open,0.00000
1,close,0.00000
2,high,0.00000
3,low,0.00000
4,volume,0.00000
5,symbol,0.00000
6,DX,0.00000
7,CORREL,0.00000
8,HT_PHASOR1,0.00000
9,SUB,347.00000


# Train and Save Agent

In [5]:
log_dir = path+"best_models\\"
if os.path.exists(log_dir) == True:
    pass
else:
    os.makedirs(log_dir) 

In [6]:
torch.cuda.set_device(1)
rounds = 2
log_dir = path+"best_models\\"
serial_num = 'abcddef'
window_size = 100
num_indicators = 30
tmp_train_obs = train_obs[:,-window_size:,:num_indicators]

env = TradingEnv_2(obs=tmp_train_obs, price=train_price, symbols=train_symbols, initial_fund=15000, 
                   pos_factor=1, both_pos=True, short_only=False, long_only=False, num_stock=stock_num, 
                   commission_rate=0.00011, valid_folder=None, mimimum_share=1, slippage=2, 
                   commission=None, test=False, valid=False, active_shares=False, cash_out_stop=False, 
                   returns=2, calculate_fund=False)
rounds_of_train = (train_obs.shape[0])*rounds
policy_kwargs = dict(net_arch=[dict(pi=[512]*15, vf=[512]*15)])

model = A2C(policy="MlpPolicy", env=env, verbose=1, seed=123456, device='cuda', policy_kwargs=policy_kwargs)
model.learn(total_timesteps=rounds_of_train, log_interval=None)
model.save(log_dir+"best_model_"+serial_num+"_"+str(window_size)+'_'+str(num_indicators))

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------This Round Info------------------
Active Shares:  False , Trading Option:  Both Position
Long Trades:  137 , Short Trades:  153
Step Index:  8130
Initial Account Balance:  15000
End Account Balance:  163.87519999999736
Rate of Return:  -0.9890749866666669
Sharpe Ratio:  -1.4762105325809627
MaxdrawDown:  0.9893640444964872
------------------This Round Ends------------------
------------------This Round Info------------------
Active Shares:  False , Trading Option:  Both Position
Long Trades:  0 , Short Trades:  29
Step Index:  8130
Initial Account Balance:  15000
End Account Balance:  22844.2626
Rate of Return:  0.5229508399999999
Sharpe Ratio:  0.8789501674765215
MaxdrawDown:  0.4285551113596207
------------------This Round Ends------------------
